Ej. 2. Experimentación con variantes de BM25

Primero instalamos todo el paquete BM25s ejecutando la siguiente orden:

In [ ]:
!pip install bm25s[full]

Ahora importamos las librerías Python necesarias para trabajar

In [ ]:
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus

Descargamos la colección TREC-COVID

In [ ]:
!gdown 1Q1ZnHkstIIQQF8NTPyKQH7sEcpVfThFO
!unzip -o trec-covid-RI.zip

Seguimos con la preparación del corpus

In [ ]:
# Parseamos el corpus
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_lines = f.readlines() # Leemos todas las lineas del JSONL
    corpus_content = [] # Preparamos una lista de objetos JSON
    for linea in corpus_lines: # Cada línea de un JSONL es un objeto JSON...
        corpus_content.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Preparamos el corpus en los dos formatos requeridos
corpus_verbatim = list() # Una versión literal que mantenga la estructura original de los "documentos".
corpus_plaintext = list() # Una versión de texto plano para tokenización e indexación.
for entry in corpus_content:
    # BM25S expects 'id', 'title', and 'text'
    document = {"id": entry["_id"], "title": entry["title"].lower(), "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

Creamos un stemmer

In [ ]:
stemmer = Stemmer.Stemmer("english")

Y ya podemos empezar el ejercicio con la implementación del código que nos permite cargar las queries

In [ ]:
# Enviamos consultas
queries = []
with open("queries.jsonl", "r", encoding="utf-8") as f:
    queries_lines = f.readlines() # Leemos todas las lineas del JSONL
    for linea in queries_lines: # Cada línea de un JSONL es un objeto JSON...
        queries.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

Cargar los juicios de relevancia

In [ ]:
def cargarJuiciosDeRelevancia(recursoJuiciosRelevancia):
    # Leer los datos del archivo de texto
    juicios = {}
    saltarLinea = True
    with open(recursoJuiciosRelevancia, 'r') as f:
        for linea in f:
            if saltarLinea:
                saltarLinea = False
                continue
            if linea.strip():  # Saltar líneas vacías
                query_id, corpus_id, score = linea.split()
                if query_id not in juicios:
                    juicios[query_id] = []
                juicios[query_id].append({"corpus_id": corpus_id, "score": int(score)})

    # El formato de los juicios de relevancia necesita ser ligeramente
    # transformado
    relevance_judgements_reformat = dict()
    for query_id in juicios:
        rel_docs = []
        for j in juicios[query_id]:
            rel_docs.append(j["corpus_id"])
        relevance_judgements_reformat[query_id] = rel_docs

    return relevance_judgements_reformat

juiciosRelevancia = cargarJuiciosDeRelevancia("qrels.tsv")

Preparamos código necesario para la generacion de las rondas de resultados

In [ ]:
def getNombreFicheroResultadoAGrabar(indice):
  match indice:
    case 0:
      return "robertson-NON-stopwords-NON-stemming"
    case 1:
      return "atire-NON-stopwords-NON-stemming"
    case 2:
      return "bm25l-NON-stopwords-NON-stemming"
    case 3:
      return "bm25plus-NON-stopwords-NON-stemming"
    case 4:
      return "lucene-NON-stopwords-NON-stemming"
    case 5:
      return "robertson-NON-stopwords-stemming"
    case 6:
      return "atire-NON-stopwords-stemming"
    case 7:
      return "bm25l-NON-stopwords-stemming"
    case 8:
      return "bm25plus-NON-stopwords-stemming"
    case 9:
      return "lucene-NON-stopwords-stemming"
    case 10:
      return "robertson-stopwords-NON-stemming"
    case 11:
      return "atire-stopwords-NON-stemming"
    case 12:
      return "bm25l-stopwords-NON-stemming"
    case 13:
      return "bm25plus-stopwords-NON-stemming"
    case 14:
      return "lucene-stopwords-NON-stemming"
    case 15:
      return "robertson-stopwords-stemming"
    case 16:
      return "atire-stopwords-stemming"
    case 17:
      return "bm25l-stopwords-stemming"
    case 18:
      return "bm25plus-stopwords-stemming"
    case 19:
      return "lucene-stopwords-stemming"


def grabarResultados(resultados, nombreFichero, map, mar, maf, mip, mir, mif):
  with open(nombreFichero + ".json", "w") as f:
    #print(resultados)
    resultados["macro_average_precision"] = map
    resultados["macro_average_recall"] = mar
    resultados["macro_average_f1"] = maf
    resultados["micro_average_precision"] = mip
    resultados["micro_average_recall"] = mir
    resultados["micro_average_f1"] = mif
    json.dump(resultados, f)


# Function to computer precision, recall and F1-score both micro- and macro-
# averaged
def compute_precision_recall_f1(run, relevance_judgements):
    # Initialize lists to hold precision, recall, and f1 scores for each query
    precision_values = []
    recall_values = []
    f1_values = []

    # Initialize global counts for micro-averaging
    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    # Compute precision, recall, and F1 score for each query
    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements[query_id]
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        # Update global counts
        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        # Compute precision and recall
        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        # Compute F1 score if both precision and recall are non-zero
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        # Append precision and recall for the current query
        precision_values.append(precision)
        recall_values.append(recall)

    # Compute macro-averages
    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    macro_average_precision = round(macro_average_precision,3)
    macro_average_recall = round(macro_average_recall,3)
    macro_average_f1 = round(macro_average_f1,3)

    # Compute micro-averages
    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    micro_average_precision = round(micro_average_precision,3)
    micro_average_recall = round(micro_average_recall,3)
    micro_average_f1 = round(micro_average_f1,3)

    return macro_average_precision, macro_average_recall, macro_average_f1, micro_average_precision, micro_average_recall, micro_average_f1


# Función para cargar consultas desde un fichero json para enviar a una
# colección dada y obtener listas de resultados.
def submit_queries_and_get_run(queries, stemmer, retriever, stopwordsConfig, max_results=100):
    run = {}

    # Iteramos para enviar cada consulta al retriever para obtener
    # resultados
    for query in queries:
        # Para evaluar después el rendimiento, es útil conservar el id de
        # la consulta
        query_id = query["_id"]

        query_string = query["metadata"]["narrative"].lower()

        # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
        # con la misma configuración que usamos al indexar el corpus.
        #
        query_tokenized = bm25s.tokenize(query_string, stopwords=stopwordsConfig, stemmer=stemmer, show_progress=False)

        # Retornamos los k resultados más “top” como una tupla con nombre.
        # Por favor, lee la documentación de BM25S para otras alternativas.
        #
        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        # Los documentos y sus puntuaciones se guardan en dos campos
        # diferentes de la tupla.
        #
        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_ids = []
        for i in range(len(returned_documents)):
##            print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            returned_ids.append(str(returned_documents[i]["id"]))
        run[query_id] = returned_ids

    return run

Y procedemos con la generación de las rondas de resultados con los 100 mejores resultados obtenidos con cada query.

In [ ]:
generacionRondas = True # cambiar este valor para realizar la generación de las rondas

eliminacionPalabrasVacias = [None, "en"]
aplicacionStemmer = [None, stemmer]
variantes = ["robertson", "atire", "bm25l", "bm25+", "lucene"]
if generacionRondas:
  indiceNombreFicheroResultado = 0

  for configPalabrasVacias in eliminacionPalabrasVacias:
    for configStemmer in aplicacionStemmer:
      for configBM25 in variantes:
        bm25_flavor = configBM25
        idf_flavor = configBM25

        print("Generando:",getNombreFicheroResultadoAGrabar(indiceNombreFicheroResultado),".json")

        corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords=configPalabrasVacias, stemmer=configStemmer, show_progress=True)
        retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor) # creamos el retriever
        retriever.index(corpus_tokenized, show_progress=True)

        resultados = submit_queries_and_get_run(queries, configStemmer, retriever, configPalabrasVacias)
        map, mar, maf, mip, mir, mif = compute_precision_recall_f1(resultados, juiciosRelevancia)

        grabarResultados(resultados, getNombreFicheroResultadoAGrabar(indiceNombreFicheroResultado), map, mar, maf, mip, mir, mif)
        indiceNombreFicheroResultado = indiceNombreFicheroResultado + 1

Cargamos los ficheros generados con cada ejecución del bloque de código anterior

In [ ]:
!gdown 1CryOYlwA0KMahDg5b9-I_qhAtbGusMMv
!unzip -o JSONs_generados.zip

Mostramos los resultados obtenidos de cada ejecución procesando los ficheros cargados antes

In [ ]:
numeroFicherosGenerados = len(eliminacionPalabrasVacias) * len(aplicacionStemmer) * len(variantes)

for i in range(numeroFicherosGenerados):
  nombreEjecucion = getNombreFicheroResultadoAGrabar(i)
  with open(nombreEjecucion + ".json", "r") as f:
    print("------Ejecución:", nombreEjecucion,"------")
    resultados = json.load(f)
    print("----Resultados----")
    print("--Queries--")
    contador = 0
    for query in resultados:
      print("Query:",query, "\tResultados:", resultados[query])
      contador = contador + 1
      if contador == len(queries):
        break
    print("--Rendimiento--")
    print("Macro average precision:", resultados["macro_average_precision"])
    print("Macro average recall:", resultados["macro_average_recall"])
    print("Macro average F1:", resultados["macro_average_f1"])
    print()
    print("Micro average precision:", resultados["micro_average_precision"])
    print("Micro average recall:", resultados["micro_average_recall"])
    print("Micro average F1:", resultados["micro_average_f1"])
    print()
    f.close()